## 동별 세대수 가져오기
+ 작성: 김지훈

### 1. 지정한 년월에 사용 가능한 최신의 법정동, 행정동 코드 자료를 테이블에서 가져온다.

In [31]:
#라이브러리 설치

import pandas as pd
# from pandas.io.json import json_normalize
import requests
import os
from tqdm.notebook import tqdm
import time
import pprint
import json
import pymysql
import csv
from urllib.parse import quote
from sqlalchemy import create_engine
import sqlalchemy
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import calendar

In [59]:
#추출하고자 하는 년월
input_year = 2023
input_month = 6
month = '202306'   # yyyymm 형식
last_day = calendar.monthrange(input_year, input_month)[1]
# print(year, month, last_day)
month_end_date = datetime(input_year, input_month, last_day).strftime("%Y%m%d")

In [48]:
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

In [49]:
# 해당 년월에 사용가능한, 저장된 데이터의 마지막 일자 가져오기
conn = dbconnect.db_connect("DEMO_DM")
cur = conn.cursor()

query = "SELECT 업데이트일자 FROM city WHERE 업데이트일자 <= '" + month_end_date + "' ORDER BY 업데이트일자 DESC LIMIT 1"
cur.execute(query)
row = cur.fetchone()
stored_date = row[0]     

conn.close()

In [50]:
# city 테이블에서 데이터 읽어오기
# conn = engine.connect()
query = "SELECT * FROM city WHERE 업데이트일자 = '" + stored_date + "'"
# result = pd.read_sql_query(query, conn)
# engine = get_engine()
engine = dbconnect.db_engine("DEMO_DM")
result = pd.read_sql(query, engine)

In [51]:
df_code_table = result.copy()
df_code_table

,업데이트일자,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명
0,20230611,2700000000,대구광역시,None,None,2700000000,대구광역시
1,20230611,2711000000,대구광역시,중구,None,2711000000,중구
2,20230611,2711051700,대구광역시,중구,동인동,2711010100,동인동1가
3,20230611,2711051700,대구광역시,중구,동인동,2711010200,동인동2가
4,20230611,2711051700,대구광역시,중구,동인동,2711010300,동인동3가
...,...,...,...,...,...,...,...
691,20230611,4785032000,경상북도,칠곡군,동명면,4785032034,남원리
692,20230611,4785032000,경상북도,칠곡군,동명면,4785032035,득명리
693,20230611,4785032000,경상북도,칠곡군,동명면,4785032036,기성리
694,20230611,4785032000,경상북도,칠곡군,동명면,4785032037,구덕리


### 2. 세대수 데이터 추출을 위한 법정동 코드 전처리

Open API를 사용하기 위해 **법정동 코드를 전처리**하는 과정입니다.  
- 해당 API는 법정동코드를 입력하였을 때, 값이 나오도록 되어 있습니다.  
- API사용 할 때, **구, 시, 리**의 법정동 코드를 제거하고 **동,읍,면** 법정동 코드만 남겨둬야합니다. 

법정동 코드는 총 10자리로 10자리의 구성은 아래와 같습니다.  
 - 시도(2) + 시군구(3) + 읍면동(3) + 리(2)

In [52]:
# 전제 코드 데이터
df_code_table_pre = df_code_table.copy()
df_code_table_pre.shape

(696, 7)

In [53]:
# '시군구' 단위 및 '읍면동' 단위의 데이터만 남겨놓는다
df_code_table_pre = df_code_table_pre[df_code_table_pre['법정동코드'].str.endswith('00')]
# '시군구' 단위 데이터 삭제
idx = df_code_table_pre[df_code_table_pre['법정동코드'].str.endswith('00000')].index
df_code_table_pre.drop(idx, inplace=True)
# 최종적으로 '읍면동' 코드만 남는다.
df_code_table_pre.shape

(344, 7)

In [54]:
#구 별로 법정동코드를 추출하여 list형식으로 변환
def make_code_list_for_daegu(df,gu_name):
    df_gu= df[df['시군구명']==gu_name]
    list_gu=  df_gu['법정동코드']
    list_gu = list_gu.unique()
    return list_gu

In [55]:
# 각 도시별 법정동 코드 불러오기

#대구광역시
df_daegu_code = df_code_table_pre[df_code_table_pre['시도명'].str.contains('대구광역시')]
df_daegu_code= df_daegu_code.dropna(subset=['읍면동명']) #null값 제거
idx = df_daegu_code[df_daegu_code['법정동코드'].str[-4:] == "0000"].index #리 값 제거
df_daegu_code.drop(idx , inplace=True)

list_daegu_cd =  df_daegu_code['법정동코드']
list_daegu_cd = list_daegu_cd.unique() #대구광역시의 법정동코드

#중간에 오류가 날 수 있어 구별로 나누어 작업 진행
list_daegu_nam_gu_cd =  make_code_list_for_daegu(df_daegu_code,"남구")
list_daegu_dalseo_gu_cd =  make_code_list_for_daegu(df_daegu_code,"달서구")
list_daegu_dalseong_gun_cd =  make_code_list_for_daegu(df_daegu_code,"달성군")
list_daegu_dong_gu_cd =  make_code_list_for_daegu(df_daegu_code,"동구")
list_daegu_buk_gu_cd =  make_code_list_for_daegu(df_daegu_code,"북구")
list_daegu_seo_gu_cd =  make_code_list_for_daegu(df_daegu_code,"서구")
list_daegu_suseong_gu_cd =  make_code_list_for_daegu(df_daegu_code,"수성구")
list_daegu_jung_gu_cd =  make_code_list_for_daegu(df_daegu_code,"중구")

#경산시
df_gyeongsan_code= df_code_table_pre[df_code_table_pre['시군구명']=='경산시'] 
df_gyeongsan_code= df_gyeongsan_code.dropna(subset=['읍면동명'])
list_gyeongsan_cd =  df_gyeongsan_code['법정동코드']
list_gyeongsan_cd = list_gyeongsan_cd.unique()#유일한 값만 남겨둠

#고령군
df_goryeong_code= df_code_table_pre[df_code_table_pre['시군구명']=='고령군']
df_goryeong_code= df_goryeong_code.dropna(subset=['읍면동명'])
list_goryeong_cd =  df_goryeong_code['법정동코드']
list_goryeong_cd = list_goryeong_cd.unique()#유일한 값만 남겨둠

#칠곡군 동명면
df_dongmyeong_code= df_code_table_pre[df_code_table_pre['읍면동명']=='동명면']
list_dongmyeong_cd =  df_dongmyeong_code['법정동코드']
list_dongmyeong_cd = list_dongmyeong_cd.unique()#유일한 값만 남겨둠


In [56]:
#개수 확인 (23년 6월 16일 기준으로 주석 작성)
daegu_len = len(list_daegu_nam_gu_cd)+len(list_daegu_dalseo_gu_cd)+len(list_daegu_dalseong_gun_cd)+len(list_daegu_dong_gu_cd)+len(list_daegu_buk_gu_cd)+len(list_daegu_seo_gu_cd)+len(list_daegu_suseong_gu_cd)+len(list_daegu_jung_gu_cd)
print("대구 : ",daegu_len) #204
print("경산 : ", len(list_gyeongsan_cd)) #36
print("고령 : ", len(list_goryeong_cd)) #8
print("칠곡군 동명면 : ", len(list_dongmyeong_cd)) #1

대구 :  204
경산 :  36
고령 :  8
칠곡군 동명면 :  1


### 3. Open API를 이용한 세대수 데이터 추출

In [57]:
import warnings
from time import sleep
warnings.filterwarnings(action='ignore')

In [67]:
service_url = 'http://apis.data.go.kr/1741000/stdgPpltnHhStus'
# 일반 인증키(Encoding)	
api_key = 'IVgu%2FZBjA6hpLryyEOpySC2RhogOhaJIUqlXN8Uyj3Gxw4s3dX0qMxfgXMTLl60%2Fs2EYAMUsyyzTqwVOnjoIhg%3D%3D'
level = '4'     # 조회결과구분. 광역시도 단위 : 1, 시군구 단위 : 2, 읍면동 단위 : 3, 읍면동 통반 단위 : 4(기본값 : 4)
reg_code = '1'  # 등록구분. 전체:1, 거주자:2, 거주불명자:3, 재외국민:4(기본값 : 1)
req_type = 'json'   # 타입. XML, JSON(기본값 : XML)
pageRows = '8000'    # 페이지 크기. 페이지당 목록 수(1~100)(기본값 : 10)
pageNo = '1'        # 페이지 번호. 기본값 : 1

In [87]:
#법정동 코드를 이용한 행정동/법정동별 세대수를 API로 구하는 함수
def make_hh_data(df,list_cd,month):
    for i in tqdm(list_cd): 
        url = f'{service_url}/selectStdgPpltnHhStus?serviceKey={api_key}&stdgCd={i}&srchFrYm={month}&srchToYm={month}&lv={level}&regSeCd={reg_code}&type={req_type}&numOfRows={pageRows}&pageNo={pageNo}'
        # url = 'https://apis.data.go.kr/1741000/stdgPpltnHhStus/selectStdgPpltnHhStus?serviceKey=IVgu%2FZBjA6hpLryyEOpySC2RhogOhaJIUqlXN8Uyj3Gxw4s3dX0qMxfgXMTLl60%2Fs2EYAMUsyyzTqwVOnjoIhg%3D%3D&stdgCd='+i+'&srchFrYm='+month+'&srchToYm='+month+'&lv=4&regSeCd=1&type=json&numOfRows=8000&pageNo=1'
        response = requests.get(url,verify=False)
        #데이터 값 출력해보기
        contents = response.text
        json_ob = json.loads(contents)
        body = json_ob['Response']['items']['item']
        dataframe = pd.json_normalize(body)
        df = pd.concat([df,dataframe],axis = 0) #데이터 결합
    return df

#시군구별 세대수를 알아보는 함수 >> 시군구 자료와 데이터 비교
def know_hhcnt(df):
    df['hhCnt'] =df['hhCnt'].astype(str).astype(int)
    df['femlNmprCnt'] =df['femlNmprCnt'].astype(str).astype(int)
    df['totNmprCnt'] =df['totNmprCnt'].astype(str).astype(int)
    df['maleNmprCnt'] =df['maleNmprCnt'].astype(str).astype(int)
    know_hhcnt = df.groupby('sggNm').sum()
    return know_hhcnt

#기존의 표와 같이 보기 편하게 순서 변경 및 컬럼 변경
def name_sort_change(df):
    #이름 변경
    df.rename(columns={'hhCnt':'세대수','tong':'통','femlNmprCnt':'여자인구수','stdgCd':'법정동코드','maleFemlRate':'남여비율',
                       'stdgNm':'법정동명','ban':'반','totNmprCnt':'총인구수','ctpvNm':'시도명','maleNmprCnt':'남자인구수',
                       'sggNm':'시군구명','dongNm':'행정동명','hhNmpr':'세대당인구','admmCd':'행정기관코드',
                       'statsYm':'통계년월','liNm':'리명'},inplace=True)
    #기존의 표와 같이 순서 변경
    df = df[['통계년월','법정동코드','시도명','시군구명','법정동명','리명','행정기관코드','행정동명','통',
             '반','총인구수','세대수','세대당인구','남자인구수','여자인구수','남여비율']]
    df['업데이트일자'] = datetime.now().strftime("%Y%m%d")
    return df

In [69]:
num_retry = 5
남구_성공 = 0
while 남구_성공 < 1:
    try:
        df_daegu_nam_gu = pd.DataFrame()
        df_daegu_nam_gu = make_hh_data(df_daegu_nam_gu,list_daegu_nam_gu_cd,month) #남구
        남구_성공 = 1
    except Exception as e:
        for retry in range(num_retry):
            try:
                print("retry")
                df_daegu_nam_gu = pd.DataFrame()
                df_daegu_nam_gu = make_hh_data(df_daegu_nam_gu,list_daegu_nam_gu_cd,month) #남구
                남구_성공 = 1
                break
            except Exception:
                continue

  0%|          | 0/3 [00:00<?, ?it/s]

In [71]:
df_daegu_nam_gu.shape

(1977, 15)

In [ ]:
num_retry = 5
달서구_성공 = 0
while 달서구_성공 < 1:
    try:
        df_daegu_dalseo_gu = pd.DataFrame()
        df_daegu_dalseo_gu = make_hh_data(df_daegu_dalseo_gu,list_daegu_dalseo_gu_cd,month) #달서구
        달서구_성공 = 1
    except Exception as e:
        for retry in range(num_retry):
            try:
                print("retry")
                df_daegu_dalseo_gu = pd.DataFrame()
                df_daegu_dalseo_gu = make_hh_data(df_daegu_dalseo_gu,list_daegu_dalseo_gu_cd,month) #달서구
                달서구_성공 = 1
                break
            except Exception:
                continue

In [72]:
df_daegu_dalseo_gu.shape

(5580, 15)

In [73]:
num_retry = 5
달성군_성공 = 0
while 달성군_성공 < 1:
    try:
        df_daegu_dalseong_gun = pd.DataFrame()
        df_daegu_dalseong_gun = make_hh_data(df_daegu_dalseong_gun,list_daegu_dalseong_gun_cd,month) #달성군

        달성군_성공 = 1
    except Exception as e:
        for retry in range(num_retry):
            try:
                print("retry")
                df_daegu_dalseong_gun = pd.DataFrame()
                df_daegu_dalseong_gun = make_hh_data(df_daegu_dalseong_gun,list_daegu_dalseong_gun_cd,month) #달성군

                달성군_성공 = 1
                break
            except Exception:
                continue

  0%|          | 0/9 [00:00<?, ?it/s]

In [74]:
num_retry = 5
동구_성공 = 0
while 동구_성공 < 1:
    try:
        df_daegu_dong_gu = pd.DataFrame()
        df_daegu_dong_gu = make_hh_data(df_daegu_dong_gu,list_daegu_dong_gu_cd,month) #동구
        동구_성공 = 1
    except Exception as e:
        for retry in range(num_retry):
            try:
                print("retry")
                df_daegu_dong_gu = pd.DataFrame()
                df_daegu_dong_gu = make_hh_data(df_daegu_dong_gu,list_daegu_dong_gu_cd,month) #동구
                동구_성공 = 1
                break
            except Exception:
                continue

  0%|          | 0/45 [00:00<?, ?it/s]

retry


  0%|          | 0/45 [00:00<?, ?it/s]

retry


  0%|          | 0/45 [00:00<?, ?it/s]

retry


  0%|          | 0/45 [00:00<?, ?it/s]

In [75]:
num_retry = 5
북구_성공 = 0
while 북구_성공 < 1:
    try:
        df_daegu_buk_gu = pd.DataFrame()
        df_daegu_buk_gu = make_hh_data(df_daegu_buk_gu,list_daegu_buk_gu_cd,month) #북구
        북구_성공 = 1
    except Exception as e:
        for retry in range(num_retry):
            try:
                print("retry")
                df_daegu_buk_gu = pd.DataFrame()
                df_daegu_buk_gu = make_hh_data(df_daegu_buk_gu,list_daegu_buk_gu_cd,month) #북구
                북구_성공 = 1
                break
            except Exception:
                continue

  0%|          | 0/31 [00:00<?, ?it/s]

retry


  0%|          | 0/31 [00:00<?, ?it/s]

retry


  0%|          | 0/31 [00:00<?, ?it/s]

In [76]:
num_retry = 5
서구_성공 = 0
while 서구_성공 < 1:
    try:
        df_daegu_seo_gu = pd.DataFrame()
        df_daegu_seo_gu = make_hh_data(df_daegu_seo_gu,list_daegu_seo_gu_cd,month) #서구
        서구_성공 = 1
    except Exception as e:
        for retry in range(num_retry):
            try:
                print("retry")
                df_daegu_seo_gu = pd.DataFrame()
                df_daegu_seo_gu = make_hh_data(df_daegu_seo_gu,list_daegu_seo_gu_cd,month) #서구
                서구_성공 = 1
                break
            except Exception:
                continue

  0%|          | 0/9 [00:00<?, ?it/s]

retry


  0%|          | 0/9 [00:00<?, ?it/s]

retry


  0%|          | 0/9 [00:00<?, ?it/s]

In [77]:
num_retry = 5
수성구_성공 = 0
while 수성구_성공 < 1:
    try:
        df_daegu_suseong_gu = pd.DataFrame()
        df_daegu_suseong_gu = make_hh_data(df_daegu_suseong_gu,list_daegu_suseong_gu_cd,month) #수성구

        수성구_성공 = 1
    except Exception as e:
        for retry in range(num_retry):
            try:
                print("retry")
                df_daegu_suseong_gu = pd.DataFrame()
                df_daegu_suseong_gu = make_hh_data(df_daegu_suseong_gu,list_daegu_suseong_gu_cd,month) #수성구

                수성구_성공 = 1
                break
            except Exception:
                continue

  0%|          | 0/26 [00:00<?, ?it/s]

In [78]:
num_retry = 5
중구_성공 = 0
while 중구_성공 < 1:
    try:
        df_daegu_jung_gu = pd.DataFrame()
        df_daegu_jung_gu = make_hh_data(df_daegu_jung_gu,list_daegu_jung_gu_cd,month) #중구
        중구_성공 = 1
    except Exception as e:
        for retry in range(num_retry):
            try:
                print("retry")
                df_daegu_jung_gu = pd.DataFrame()
                df_daegu_jung_gu = make_hh_data(df_daegu_jung_gu,list_daegu_jung_gu_cd,month) #중구
                중구_성공 = 1
                break
            except Exception:
                continue

  0%|          | 0/57 [00:00<?, ?it/s]

In [79]:
#경상북도 경산
num_retry = 5
경산_성공 = 0
while 경산_성공 < 1:
    try:
        df_gyeongsan = pd.DataFrame()
        df_gyeongsan = make_hh_data(df_gyeongsan,list_gyeongsan_cd,month) #경산

        경산_성공 = 1
    except Exception as e:
        for retry in range(num_retry):
            try:
                print("retry")
                df_gyeongsan = pd.DataFrame()
                df_gyeongsan = make_hh_data(df_gyeongsan,list_gyeongsan_cd,month) #경산

                경산_성공 = 1
                break
            except Exception:
                continue

  0%|          | 0/36 [00:00<?, ?it/s]

retry


  0%|          | 0/36 [00:00<?, ?it/s]

retry


  0%|          | 0/36 [00:00<?, ?it/s]

In [80]:
#경상북도 고령, 동명면
num_retry = 5
고령_동명면_성공 = 0
while 고령_동명면_성공 < 1:
    try:

        df_goryeong = pd.DataFrame()
        df_goryeong = make_hh_data(df_goryeong,list_goryeong_cd,month) #고령

        df_dongmyeong= pd.DataFrame()
        df_dongmyeong = make_hh_data(df_dongmyeong,list_dongmyeong_cd,month) #동명면
        
        고령_동명면_성공 = 1
    except Exception as e:
        for retry in range(num_retry):
            try:
                print("retry")

                df_goryeong = pd.DataFrame()
                df_goryeong = make_hh_data(df_goryeong,list_goryeong_cd,month) #고령
                
                df_dongmyeong= pd.DataFrame()
                df_dongmyeong = make_hh_data(df_dongmyeong,list_dongmyeong_cd,month) #동명면
                

                고령_동명면_성공 = 1
                break
            except Exception:
                continue

  0%|          | 0/8 [00:00<?, ?it/s]

retry


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [81]:
if(남구_성공==1 & 달서구_성공==1 & 달성군_성공==1 & 동구_성공==1 & 북구_성공==1
  & 서구_성공==1 & 수성구_성공==1 & 중구_성공==1 & 경산_성공==1 & 고령_동명면_성공==1) :
    print("모두 성공")
    
elif (남구_성공 != 1):
    print("남구 실패")
        
elif (달서구_성공 != 1):
    print("달서구 실패")
    
elif (달성군_성공 != 1):
    print("달성군 실패")
        
elif (동구_성공 != 1):
    print("동구 실패")
    
elif (북구_성공 != 1):
    print("북구 실패")
        
elif (서구_성공 != 1):
    print("서구 실패")
    
elif (수성구_성공 != 1):
    print("수성구 실패")
        
elif (중구_성공 != 1):
    print("중구 실패")
    
elif (경산_성공 != 1):
    print("경산 실패")
        
elif (고령_동명면_성공 != 1):
    print("고령, 동명면 실패")
    

모두 성공


In [108]:
df_daegu = pd.concat([df_daegu_nam_gu,df_daegu_dalseo_gu,df_daegu_dalseong_gun,df_daegu_dong_gu,df_daegu_seo_gu,df_daegu_suseong_gu,df_daegu_buk_gu,df_daegu_jung_gu],axis = 0)
df_daegu.drop_duplicates(keep = 'first',inplace=True) #중복값 제거 >> 중복값이 있을 때 첫번째 값만 남겨둠
df_gyeongsang = pd.concat([df_gyeongsan,df_dongmyeong,df_goryeong],axis = 0)
df_gyeongsang.drop_duplicates(keep = 'first',inplace=True) #중복값 제거 >> 중복값이 있을 때 첫번째 값만 남겨둠
print('대구 데이터 길이 : ',len(df_daegu))
print('경상북도 데이터 길이 : ',len(df_gyeongsang))
df_house_hold = pd.concat([df_daegu,df_gyeongsang],axis = 0) # 도시별 데이터 결합
print("총 데이터 길이 : ",len(df_house_hold))
df_house_hold.head(5)

대구 데이터 길이 :  24825
경상북도 데이터 길이 :  2766
총 데이터 길이 :  27591


,hhCnt,tong,femlNmprCnt,stdgCd,maleFemlRate,stdgNm,ban,totNmprCnt,ctpvNm,maleNmprCnt,sggNm,dongNm,hhNmpr,admmCd,statsYm,liNm
0,9,1,6,2720010100,0.83,이천동,1,11,대구광역시,5,남구,이천동,1.22,2720051500,202306,NaN
1,25,1,24,2720010100,0.96,이천동,2,47,대구광역시,23,남구,이천동,1.88,2720051500,202306,NaN
2,35,1,26,2720010100,1.12,이천동,4,55,대구광역시,29,남구,이천동,1.57,2720051500,202306,NaN
3,50,1,34,2720010100,1.03,이천동,5,69,대구광역시,35,남구,이천동,1.38,2720051500,202306,NaN
4,55,2,70,2720010100,0.89,이천동,1,132,대구광역시,62,남구,이천동,2.40,2720051500,202306,NaN


In [109]:
print(know_hhcnt(df_house_hold))

        hhCnt                                               tong  femlNmprCnt  \
sggNm                                                                           
경산시    129053  1234567891011121335361415153737373737373738383...       132794   
고령군     16906  1111111112222222223262645678889101112131415161...        14675   
남구      76592  1111222223333344444455555566666667777778888888...        73736   
달서구    236791  1111112222223333334444455555566666677777788888...       271747   
달성군    113996  1111111111112222222222233333333344444444555555...       129547   
동구     160720  1111111111111111111111111111111222222222222222...       173316   
북구     192580  1141919191919202020202020212121212111111222222...       214649   
서구      81819  1111111111111111111112222222222222222222222333...        80377   
수성구    171299  1111111111111111111111111111122222222222222222...       211396   
중구      42582  1111122333334455554455162323232323232366666677...        43725   
칠곡군      2969  1111122222333

In [110]:
#데이터 컬럼명 변경 및 위치 변경
df_house_hold = name_sort_change(df_house_hold)

In [112]:
# NaN 값을 공백으로 변환 (Table에 저장하기 위함)
df_house_hold['리명'] = df_house_hold['리명'].fillna('')

In [113]:
df_house_hold.head()

,통계년월,법정동코드,시도명,시군구명,법정동명,리명,행정기관코드,행정동명,통,반,총인구수,세대수,세대당인구,남자인구수,여자인구수,남여비율,업데이트일자
0,202306,2720010100,대구광역시,남구,이천동,,2720051500,이천동,1,1,11,9,1.22,5,6,0.83,20230711
1,202306,2720010100,대구광역시,남구,이천동,,2720051500,이천동,1,2,47,25,1.88,23,24,0.96,20230711
2,202306,2720010100,대구광역시,남구,이천동,,2720051500,이천동,1,4,55,35,1.57,29,26,1.12,20230711
3,202306,2720010100,대구광역시,남구,이천동,,2720051500,이천동,1,5,69,50,1.38,35,34,1.03,20230711
4,202306,2720010100,대구광역시,남구,이천동,,2720051500,이천동,2,1,132,55,2.40,62,70,0.89,20230711


# 5. 법정동, 행정동 세대수 데이터  SQL에 삽입 

In [114]:
conn = dbconnect.db_connect("DEMO_DW")
cursor = conn.cursor()

query = f'SELECT EXISTS (SELECT * FROM household_dong WHERE 통계년월 = {month})'

cursor.execute(query)
row = cursor.fetchone()
data_exist = row[0]     # 저장된 데이터의 유무(1 - 데이터 있음)

conn.close()

In [115]:
if data_exist == 1:      # 데이터 존재하면
    print(f'{month} 에 데이터가 존재합니다.')
else:
    conn = dbconnect.db_connect("DEMO_DW")
    cur = conn.cursor()

    for row in df_house_hold.itertuples():
        sql = "insert into household_dong (통계년월, 법정동코드, 시도명, 시군구명, 법정동명, 리명, \
                                           행정기관코드, 행정동명, 통, 반, 총인구수, 세대수, 세대당인구, \
                                           남자인구수, 여자인구수, 남여비율, 업데이트일자) \
               values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12], row[13], row[14], row[15], row[16], row[17]))

    conn.commit()
    print(f'{month} 데이터를 저장하였습니다.')
    conn.close()

202306 데이터를 저장하였습니다.
